# Data Gathering, Scraping, Munging and Cleaning for StockAssist

## Introduction
Here, we are searching for authentic sources of data which will be useful for analyzing stock market data. These sources need to be verified in order to generate an accurate analysis upon the stocks. First of all, these multiple data sources need to be scraped in the format available. Once we obtain all the data from multiple data sources, this raw data is supposed to be refined into content or formats better suited for consumption for our database. Finally, this data is cleaned by detecting corrupt or inaccurate records from the dataset and then replacing, modifying or deleting this coarse data. 

## Data Scraping
Data Scraping is a technique used to extract data from the internet into a file or spreadsheet. We will be scraping data from the following three sources:
- Data Scraping from a CSV File
- Web Scraping
- Data Scraping using an API

### Data Scraping from a CSV File

In [1]:
import pandas as pd
import mysql.connector # Library required for connecting to the MySQL Database

In [2]:
# Connecting to the MySQL Database
mydb = mysql.connector.connect(host='localhost', user = 'root', passwd = 'Northeastern@NEU10', database = 'stockassist')
mycursor = mydb.cursor()

In [3]:
# Read data from the CSV file and insert it into a DataFrame
csv_df = pd.read_csv('Ticker_Master.csv')

In [4]:
# Display data in the DataFrame
csv_df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2013 (1888)
4,ABMD,Abiomed,Health Care,Health Care Equipment,"Danvers, Massachusetts",1981


### Web Scraping
We will be performing web scraping using Beautiful Soup which is a Python library for extracting data from HTML and XML files. It works with the parser to provide idiomatic ways of navigating, searching and modifying the parse tree.

In [5]:
import requests # Library required to send HTTP requests
from bs4 import BeautifulSoup # Library required for performing web scraping using Beautiful Soup

In [6]:
# Sending HTTP request to the webpage which is supposed to be scraped
request = requests.get('https://www.marketbeat.com/types-of-stock/sp-500-stocks/')

In [7]:
# Connecting to the webpage via Beautiful Soup using the HTML parser
soup = BeautifulSoup(request.content, 'html.parser')

In [8]:
# Converting Beautiful Soup parse tree into a nicely formatted Unicode string
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <title>
   S&amp;P 500 Stocks | List of Companies
  </title>
  <meta charset="utf-8"/>
  <meta content="width=device-width, height=device-height, initial-scale=1.0" name="viewport"/>
  <!--[if IE]><meta http-equiv="X-UA-Compatible" content="IE=edge" /><![endif]-->
  <meta content="21tyd2NTsebvto7W56gttKaElQtkak0Ld32xGvr_COs" name="google-site-verification"/>
  <meta content="43C334F81BEB9DE14F872D6C91B4FC29" name="msvalidate.01"/>
  <meta content="63e01d3704030089" name="y_key"/>
  <meta content="1450892295227696" property="fb:pages"/>
  <meta content="#24778c" name="theme-color"/>
  <meta content="app-id=917482225" name="apple-itunes-app"/>
  <meta content="max-image-preview:large" name="robots"/>
  <link href="https://www.marketbeat.com/Sitemap.ashx" rel="sitemap" title="Sitemap" type="application/xml"/>
  <link href="https://www.marketbeat.com/rss.ashx?type=headlines" rel="alternate" title="MarketBeat Headlines RSS" type="application/rss+xm

In [9]:
# Searching for 'table' tags inside the Unicode string
table = soup.find('table', attrs = {'class':'scroll-table sort-table'})
table

<table class="scroll-table sort-table"><thead><tr><th data-clean="Symbol|Company" data-sort="string">Company</th><th data-clean="Current Price|Price Change" data-sort="float">Current Price</th><th data-sort="float">PE Ratio</th><th data-sort="float">Market Cap</th><th data-sort="float">Volume</th><th data-sort="float">Average Volume</th><th data-sort="string">Indicator(s)</th></tr></thead><tbody><tr><td data-clean="AAPL|Apple"><a class="nounderline" href="/stocks/NASDAQ/AAPL/"><div class="company-thumbnail"><img alt="Apple Inc. stock logo" height="53" loading="lazy" src="https://www.marketbeat.com/logos/thumbnail/apple-inc-logo.png" width="80"/></div><div class="ticker-area">AAPL</div><div class="title-area">Apple</div></a></td><td data-clean="$146.01|-1.2%">$146.01<br/><span class="c-red">-1.2%</span></td><td data-sort-value="23.8968903436989">23.90</td><td data-sort-value="2322744.163">$2.32 trillion</td><td data-sort-value="2436586">2.44 million</td><td data-sort-value="88113914">88

In [10]:
# Initialize lists to hold the Table details
headers = []
data = []

In [11]:
# Fetching headers using the 'th' tags inside the 'tr' tags
table_headers = soup.thead
for x in table_headers.findAll('tr'):
    for y in x.findAll('th'):
        headers.append(y.text)

In [12]:
headers

['Company',
 'Current Price',
 'PE Ratio',
 'Market Cap',
 'Volume',
 'Average Volume',
 'Indicator(s)']

In [13]:
# Fetching table data using the 'td' tags inside the 'tr' tags
table_body = soup.tbody
for i in table_body.findAll('tr')[1:]:
    td_tags = i.findAll('td')
    td_val = [j.get_text(separator = ',') for j in td_tags]
    data.append(td_val)

In [14]:
data

[['MSFT,Microsoft',
  '$248.92,-2.4%',
  '26.82',
  '$1.86 trillion',
  '923,843',
  '31.44 million',
  'Dividend Announcement,Insider Selling'],
 ['GOOG,Alphabet',
  '$99.60,-1.2%',
  '19.78',
  '$1.29 trillion',
  '814,487',
  '29.00 million',
  'Short Interest ↑,Analyst Revision'],
 ['GOOGL,Alphabet',
  '$99.25,-1.2%',
  '19.71',
  '$1.28 trillion',
  '987,159',
  '35.08 million',
  'Analyst Revision'],
 ['AMZN,Amazon.com',
  '$91.08,-3.2%',
  '83.60',
  '$929.17 billion',
  '2.08 million',
  '75.93 million',
  ''],
 ['TSLA,Tesla',
  '$181.03,-7.1%',
  '55.93',
  '$571.65 billion',
  '1.95 million',
  '82.05 million',
  'Gap Down'],
 ['UNH,UnitedHealth Group',
  '$535.73,-0.1%',
  '26.24',
  '$500.56 billion',
  '101,478',
  '3.13 million',
  'Ex-Dividend,Analyst Revision'],
 ['JNJ,Johnson & Johnson',
  '$178.87,0.0%',
  '24.91',
  '$467.65 billion',
  '278,497',
  '7.32 million',
  'Insider Selling'],
 ['XOM,Exxon Mobil',
  '$106.67,-2.9%',
  '8.71',
  '$439.30 billion',
  '792,399

In [15]:
# Inserting the Table data with the headers inside the DataFrame
web_df = pd.DataFrame(data, columns = headers)

In [16]:
# Display the DataFrame after Web Scraping
web_df.head()

,Company,Current Price,PE Ratio,Market Cap,Volume,Average Volume,Indicator(s)
0,"MSFT,Microsoft","$248.92,-2.4%",26.82,$1.86 trillion,"923,843",31.44 million,"Dividend Announcement,Insider Selling"
1,"GOOG,Alphabet","$99.60,-1.2%",19.78,$1.29 trillion,"814,487",29.00 million,"Short Interest ↑,Analyst Revision"
2,"GOOGL,Alphabet","$99.25,-1.2%",19.71,$1.28 trillion,"987,159",35.08 million,Analyst Revision
3,"AMZN,Amazon.com","$91.08,-3.2%",83.60,$929.17 billion,2.08 million,75.93 million,
4,"TSLA,Tesla","$181.03,-7.1%",55.93,$571.65 billion,1.95 million,82.05 million,Gap Down


### Data Scraping using API
We will be scraping stock market prices data from the Yahoo Finance API. The Yahoo Finance API is used to obtain historical and real time data for a variety of financial markets and products. yfinance is a popular open source library which is free and easy to setup with high granularity of data.

In [17]:
import yfinance as yf # Library required for connecting to the Yahoo Finance API

In [18]:
# Generate daily OHLC data for the tickers
ticker_list = ['AAPL', 'MSFT', 'AMZN', 'TSLA']
daily_df = yf.download(ticker_list, group_by='Ticker', period='11mo', interval='1d')
daily_df = daily_df.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index(level=1)

[*********************100%***********************]  4 of 4 completed


In [19]:
daily_df.head()

,Ticker,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
2022-01-05,AAPL,173.910645,174.919998,180.169998,174.639999,179.610001,94537600
2022-01-05,AMZN,164.356995,164.356995,167.126495,164.356995,166.882996,64302000
2022-01-05,MSFT,313.442993,316.380005,326.070007,315.980011,325.859985,40054300
2022-01-05,TSLA,362.706665,362.706665,390.113342,360.336670,382.216675,80119800
2022-01-06,AAPL,171.007523,172.000000,175.300003,171.639999,172.699997,96904000


In [20]:
# Generate daily OHLC data for the tickers
ticker_list = ['AAPL', 'MSFT', 'AMZN', 'TSLA']
hourly_df = yf.download(ticker_list, group_by='Ticker', period='1d', interval='1h')
hourly_df = hourly_df.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index(level=1)

[*********************100%***********************]  4 of 4 completed


In [21]:
hourly_df.head()

,Ticker,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
2022-12-05 09:30:00-05:00,AAPL,148.480804,148.480804,150.919907,147.500000,147.770004,22202031.0
2022-12-05 09:30:00-05:00,AMZN,92.684998,92.684998,94.059998,92.510002,93.050003,16020869.0
2022-12-05 09:30:00-05:00,MSFT,251.580002,251.580002,253.819000,251.070007,252.009995,4548397.0
2022-12-05 09:30:00-05:00,TSLA,184.960007,184.960007,191.270004,183.800003,189.100006,31428674.0
2022-12-05 10:30:00-05:00,AAPL,146.880005,146.880005,148.750000,146.820007,148.479996,9532365.0


## Data Munging
Data Munging (or Data Wrangling) is the process of preparing your data for a dedicated purpose - taking the data from its raw state and transforming and mapping into another format, normally for use beyond its original intent.

### Data Munging on CSV DataFrame

In [22]:
csv_df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2013 (1888)
4,ABMD,Abiomed,Health Care,Health Care Equipment,"Danvers, Massachusetts",1981


In [23]:
# Renaming columns for further processing
csv_df=csv_df.rename(columns = {'Headquarters Location':'Location'})

In [24]:
csv_df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Location,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2013 (1888)
4,ABMD,Abiomed,Health Care,Health Care Equipment,"Danvers, Massachusetts",1981


In [25]:
# Splitting the 'Location' column into two columns for more authenticity
csv_df1 = pd.DataFrame(csv_df.Location.str.split(',',1).tolist(), columns = ['Headquarters','City'])

In [26]:
csv_df1

,Headquarters,City
0,Saint Paul,Minnesota
1,Milwaukee,Wisconsin
2,North Chicago,Illinois
3,North Chicago,Illinois
4,Danvers,Massachusetts
...,...,...
498,Louisville,Kentucky
499,Lincolnshire,Illinois
500,Warsaw,Indiana
501,Salt Lake City,Utah


In [27]:
# Replace the 'Location' column with 'Headquarters'
csv_df['Location'] = csv_df1['Headquarters']
# Insert the 'City' column
csv_df.insert(5, 'City', csv_df1['City'])

In [28]:
# Display CSV DataFrame after performing Data Munging
csv_df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Location,City,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,Saint Paul,Minnesota,1902
1,AOS,A. O. Smith,Industrials,Building Products,Milwaukee,Wisconsin,1916
2,ABT,Abbott,Health Care,Health Care Equipment,North Chicago,Illinois,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,North Chicago,Illinois,2013 (1888)
4,ABMD,Abiomed,Health Care,Health Care Equipment,Danvers,Massachusetts,1981


### Data Munging on Web Scraping DataFrame

In [29]:
web_df.head()

,Company,Current Price,PE Ratio,Market Cap,Volume,Average Volume,Indicator(s)
0,"MSFT,Microsoft","$248.92,-2.4%",26.82,$1.86 trillion,"923,843",31.44 million,"Dividend Announcement,Insider Selling"
1,"GOOG,Alphabet","$99.60,-1.2%",19.78,$1.29 trillion,"814,487",29.00 million,"Short Interest ↑,Analyst Revision"
2,"GOOGL,Alphabet","$99.25,-1.2%",19.71,$1.28 trillion,"987,159",35.08 million,Analyst Revision
3,"AMZN,Amazon.com","$91.08,-3.2%",83.60,$929.17 billion,2.08 million,75.93 million,
4,"TSLA,Tesla","$181.03,-7.1%",55.93,$571.65 billion,1.95 million,82.05 million,Gap Down


In [30]:
# Renaming columns for further processing
web_df = web_df.rename(columns = {'Current Price':'Price'})
web_df = web_df.rename(columns = {'Market Cap':'Market_Cap'})
web_df = web_df.rename(columns = {'PE Ratio':'PE_Ratio'})
web_df = web_df.rename(columns = {'Average Volume':'Average_Volume'})

In [31]:
# Splitting the 'Company' column into 'Symbol' and 'Company' columns for more authenticity
web_df1 = pd.DataFrame(web_df.Company.str.split(',',1).tolist(), columns = ['Symbol','Company'])

In [32]:
web_df1.head()

,Symbol,Company
0,MSFT,Microsoft
1,GOOG,Alphabet
2,GOOGL,Alphabet
3,AMZN,Amazon.com
4,TSLA,Tesla


In [33]:
# Splitting the 'Price' column into 'Price' and 'Price_Change_Percent' columns for more authenticity
web_df2 = pd.DataFrame(web_df.Price.str.split(',',1).tolist(), columns = ['Price','Price_Change_Percent'])
web_df2['Price'] = web_df2['Price'].str.replace('$', '')

C:\Users\Yash\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


In [34]:
web_df2.head()

,Price,Price_Change_Percent
0,248.92,-2.4%
1,99.60,-1.2%
2,99.25,-1.2%
3,91.08,-3.2%
4,181.03,-7.1%


In [35]:
web_df

,Company,Price,PE_Ratio,Market_Cap,Volume,Average_Volume,Indicator(s)
0,"MSFT,Microsoft","$248.92,-2.4%",26.82,$1.86 trillion,"923,843",31.44 million,"Dividend Announcement,Insider Selling"
1,"GOOG,Alphabet","$99.60,-1.2%",19.78,$1.29 trillion,"814,487",29.00 million,"Short Interest ↑,Analyst Revision"
2,"GOOGL,Alphabet","$99.25,-1.2%",19.71,$1.28 trillion,"987,159",35.08 million,Analyst Revision
3,"AMZN,Amazon.com","$91.08,-3.2%",83.60,$929.17 billion,2.08 million,75.93 million,
4,"TSLA,Tesla","$181.03,-7.1%",55.93,$571.65 billion,1.95 million,82.05 million,Gap Down
...,...,...,...,...,...,...,...
501,"PENN,PENN Entertainment","$34.17,-2.5%",24.06,$5.30 billion,"62,300",3.94 million,
502,"NWL,Newell Brands","$12.59,-3.1%",9.47,$5.21 billion,"225,456",3.54 million,
503,"PVH,PVH","$70.85,-2.9%",11.23,$4.62 billion,"53,347",1.44 million,"Analyst Report,Analyst Revision"
504,"VNO,Vornado Realty Trust","$22.99,-5.5%",45.98,$4.41 billion,"96,893",2.11 million,


In [36]:
# Dropping unwanted columns from the dataframe
web_df = web_df.drop(['Company', 'Price', 'Indicator(s)'], axis=1)
# Manipulating column into appropriate format for insertion
web_df['Market_Cap'] = web_df['Market_Cap'].str.replace('$', '')

C:\Users\Yash\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.


In [37]:
# Concatenate all the DataFrames into a single DataFrame
final_web_df = pd.concat([web_df1, web_df2, web_df], axis=1)

In [38]:
final_web_df

,Symbol,Company,Price,Price_Change_Percent,PE_Ratio,Market_Cap,Volume,Average_Volume
0,MSFT,Microsoft,248.92,-2.4%,26.82,1.86 trillion,"923,843",31.44 million
1,GOOG,Alphabet,99.60,-1.2%,19.78,1.29 trillion,"814,487",29.00 million
2,GOOGL,Alphabet,99.25,-1.2%,19.71,1.28 trillion,"987,159",35.08 million
3,AMZN,Amazon.com,91.08,-3.2%,83.60,929.17 billion,2.08 million,75.93 million
4,TSLA,Tesla,181.03,-7.1%,55.93,571.65 billion,1.95 million,82.05 million
...,...,...,...,...,...,...,...,...
501,PENN,PENN Entertainment,34.17,-2.5%,24.06,5.30 billion,"62,300",3.94 million
502,NWL,Newell Brands,12.59,-3.1%,9.47,5.21 billion,"225,456",3.54 million
503,PVH,PVH,70.85,-2.9%,11.23,4.62 billion,"53,347",1.44 million
504,VNO,Vornado Realty Trust,22.99,-5.5%,45.98,4.41 billion,"96,893",2.11 million


### Data Munging on API DataFrame

In [39]:
daily_df.head()

,Ticker,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
2022-01-05,AAPL,173.910645,174.919998,180.169998,174.639999,179.610001,94537600
2022-01-05,AMZN,164.356995,164.356995,167.126495,164.356995,166.882996,64302000
2022-01-05,MSFT,313.442993,316.380005,326.070007,315.980011,325.859985,40054300
2022-01-05,TSLA,362.706665,362.706665,390.113342,360.336670,382.216675,80119800
2022-01-06,AAPL,171.007523,172.000000,175.300003,171.639999,172.699997,96904000


In [40]:
hourly_df.head()

,Ticker,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
2022-12-05 09:30:00-05:00,AAPL,148.480804,148.480804,150.919907,147.500000,147.770004,22202031.0
2022-12-05 09:30:00-05:00,AMZN,92.684998,92.684998,94.059998,92.510002,93.050003,16020869.0
2022-12-05 09:30:00-05:00,MSFT,251.580002,251.580002,253.819000,251.070007,252.009995,4548397.0
2022-12-05 09:30:00-05:00,TSLA,184.960007,184.960007,191.270004,183.800003,189.100006,31428674.0
2022-12-05 10:30:00-05:00,AAPL,146.880005,146.880005,148.750000,146.820007,148.479996,9532365.0


In [41]:
# Resetting the Date index into a column
daily_df.reset_index(inplace=True)

In [42]:
daily_df.head()

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume
0,2022-01-05,AAPL,173.910645,174.919998,180.169998,174.639999,179.610001,94537600
1,2022-01-05,AMZN,164.356995,164.356995,167.126495,164.356995,166.882996,64302000
2,2022-01-05,MSFT,313.442993,316.380005,326.070007,315.980011,325.859985,40054300
3,2022-01-05,TSLA,362.706665,362.706665,390.113342,360.336670,382.216675,80119800
4,2022-01-06,AAPL,171.007523,172.000000,175.300003,171.639999,172.699997,96904000


In [43]:
# Resetting the Date index into a column
hourly_df.reset_index(inplace=True)

In [44]:
hourly_df.head()

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume
0,2022-12-05 09:30:00-05:00,AAPL,148.480804,148.480804,150.919907,147.500000,147.770004,22202031.0
1,2022-12-05 09:30:00-05:00,AMZN,92.684998,92.684998,94.059998,92.510002,93.050003,16020869.0
2,2022-12-05 09:30:00-05:00,MSFT,251.580002,251.580002,253.819000,251.070007,252.009995,4548397.0
3,2022-12-05 09:30:00-05:00,TSLA,184.960007,184.960007,191.270004,183.800003,189.100006,31428674.0
4,2022-12-05 10:30:00-05:00,AAPL,146.880005,146.880005,148.750000,146.820007,148.479996,9532365.0


## Data Cleaning
Data cleaning is the process of fixing or removing incorrect, corrupted, incorrectly formatted, duplicate or incomplete data within a dataset.

### Data Cleaning on CSV DataFrame

In [45]:
csv_df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Location,City,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,Saint Paul,Minnesota,1902
1,AOS,A. O. Smith,Industrials,Building Products,Milwaukee,Wisconsin,1916
2,ABT,Abbott,Health Care,Health Care Equipment,North Chicago,Illinois,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,North Chicago,Illinois,2013 (1888)
4,ABMD,Abiomed,Health Care,Health Care Equipment,Danvers,Massachusetts,1981


In [46]:
# Formatting the 'Founded' column to only contain YEAR data
csv_df['Founded'] = csv_df['Founded'].str[:4]

In [47]:
csv_df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Location,City,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,Saint Paul,Minnesota,1902
1,AOS,A. O. Smith,Industrials,Building Products,Milwaukee,Wisconsin,1916
2,ABT,Abbott,Health Care,Health Care Equipment,North Chicago,Illinois,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,North Chicago,Illinois,2013
4,ABMD,Abiomed,Health Care,Health Care Equipment,Danvers,Massachusetts,1981


In [48]:
# Fetching number of rows and columns
csv_df.shape

(503, 7)

In [49]:
# Fetching unique values in each column
csv_df.nunique()

Symbol               503
Security             503
GICS Sector           11
GICS Sub-Industry    122
Location             238
City                  49
Founded              163
dtype: int64

In [50]:
# Searching for duplicate entries
csv_df.duplicated().sum()

0

In [51]:
# Searching for NULL values in the DataFrame
csv_df.isnull().sum()

Symbol               0
Security             0
GICS Sector          0
GICS Sub-Industry    0
Location             0
City                 0
Founded              0
dtype: int64

In [52]:
# Displaying final DataFrame after Data Cleaning
csv_df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Location,City,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,Saint Paul,Minnesota,1902
1,AOS,A. O. Smith,Industrials,Building Products,Milwaukee,Wisconsin,1916
2,ABT,Abbott,Health Care,Health Care Equipment,North Chicago,Illinois,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,North Chicago,Illinois,2013
4,ABMD,Abiomed,Health Care,Health Care Equipment,Danvers,Massachusetts,1981


In [53]:
# Cleaning existing data from the database
csv_del_query = 'delete from ticker_master where tck_symbol is not null'
mycursor.execute(csv_del_query)
mydb.commit()

In [54]:
#Iterate into the CSV DataFrame
for i,row in csv_df.iterrows():
    #INSERT query for inserting the data into the database
    csv_sql = "INSERT INTO stockassist.ticker_master VALUES (%s,%s,%s,%s,%s,%s,%s)"
    try:
        #Execute the SQL query
        mycursor.execute(csv_sql, tuple(row))
        print('Ticker Master Record inserted')
        #Commit the executed query
        mydb.commit()
    except:
        #Display message if the record already exists inside the table
        print('Ticker Master Record already exists')

Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Mas

Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Master Record inserted
Ticker Mas

### Data Cleaning on Web Scraping DataFrame

In [55]:
final_web_df.head()

,Symbol,Company,Price,Price_Change_Percent,PE_Ratio,Market_Cap,Volume,Average_Volume
0,MSFT,Microsoft,248.92,-2.4%,26.82,1.86 trillion,"923,843",31.44 million
1,GOOG,Alphabet,99.60,-1.2%,19.78,1.29 trillion,"814,487",29.00 million
2,GOOGL,Alphabet,99.25,-1.2%,19.71,1.28 trillion,"987,159",35.08 million
3,AMZN,Amazon.com,91.08,-3.2%,83.60,929.17 billion,2.08 million,75.93 million
4,TSLA,Tesla,181.03,-7.1%,55.93,571.65 billion,1.95 million,82.05 million


In [56]:
# Fetching number of rows and columns
final_web_df.shape

(506, 8)

In [57]:
# Fetching unique values in each column
final_web_df.nunique()

Symbol                  498
Company                 495
Price                   494
Price_Change_Percent     88
PE_Ratio                452
Market_Cap              482
Volume                  501
Average_Volume          403
dtype: int64

In [58]:
# Searching for duplicate entries
final_web_df.duplicated().sum()

3

In [59]:
# Removing duplicate entries from the DataFrame
final_web_df.drop_duplicates(subset="Symbol", keep=False, inplace=True)

In [60]:
# Replacing 'N/A' value with NULL value
final_web_df['PE_Ratio'] = final_web_df['PE_Ratio'].replace('N/A',None)
final_web_df = final_web_df.where((pd.notnull(final_web_df)), None)

In [61]:
final_web_df.head()

,Symbol,Company,Price,Price_Change_Percent,PE_Ratio,Market_Cap,Volume,Average_Volume
0,MSFT,Microsoft,248.92,-2.4%,26.82,1.86 trillion,"923,843",31.44 million
1,GOOG,Alphabet,99.60,-1.2%,19.78,1.29 trillion,"814,487",29.00 million
2,GOOGL,Alphabet,99.25,-1.2%,19.71,1.28 trillion,"987,159",35.08 million
3,AMZN,Amazon.com,91.08,-3.2%,83.60,929.17 billion,2.08 million,75.93 million
4,TSLA,Tesla,181.03,-7.1%,55.93,571.65 billion,1.95 million,82.05 million


In [62]:
# Searching for NULL values in the DataFrame
final_web_df.isnull().sum()

Symbol                  0
Company                 0
Price                   0
Price_Change_Percent    1
PE_Ratio                0
Market_Cap              0
Volume                  0
Average_Volume          0
dtype: int64

In [63]:
# Displaying the 5 entries which have NULL Price Change Percentage in the DataFrame
print(final_web_df[final_web_df.isna().any(axis = 1)])

    Symbol         Company   Price Price_Change_Percent PE_Ratio  \
419   CTXS  Citrix Systems  103.90                 None    40.12   

        Market_Cap Volume Average_Volume  
419  13.18 billion     33   1.70 million  


In [64]:
# Displaying final DataFrame after Data Cleaning
final_web_df.head()

,Symbol,Company,Price,Price_Change_Percent,PE_Ratio,Market_Cap,Volume,Average_Volume
0,MSFT,Microsoft,248.92,-2.4%,26.82,1.86 trillion,"923,843",31.44 million
1,GOOG,Alphabet,99.60,-1.2%,19.78,1.29 trillion,"814,487",29.00 million
2,GOOGL,Alphabet,99.25,-1.2%,19.71,1.28 trillion,"987,159",35.08 million
3,AMZN,Amazon.com,91.08,-3.2%,83.60,929.17 billion,2.08 million,75.93 million
4,TSLA,Tesla,181.03,-7.1%,55.93,571.65 billion,1.95 million,82.05 million


In [65]:
# Cleaning existing data from the database
web_del_query = 'delete from company_details where cmd_symbol is not null'
mycursor.execute(web_del_query)
mydb.commit()

In [66]:
#Iterate into the Web DataFrame
for i,row in final_web_df.iterrows():
    #INSERT query for inserting the data into the database
    web_sql = "INSERT INTO stockassist.Company_Details VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
    try:
        #Execute the SQL query
        mycursor.execute(web_sql, tuple(row))
        print(row[0] + ' Company Details Record inserted')
        #Commit the executed query
        mydb.commit()
    except:
        #Display message if the record already exists inside the table
        print(row[0] + ' Company Details Record already exists')

MSFT Company Details Record inserted
GOOG Company Details Record inserted
GOOGL Company Details Record inserted
AMZN Company Details Record inserted
TSLA Company Details Record inserted
UNH Company Details Record inserted
JNJ Company Details Record inserted
XOM Company Details Record inserted
NVDA Company Details Record inserted
WMT Company Details Record inserted
V Company Details Record inserted
JPM Company Details Record inserted
PG Company Details Record inserted
LLY Company Details Record inserted
MA Company Details Record inserted
CVX Company Details Record inserted
HD Company Details Record inserted
META Company Details Record inserted
ABBV Company Details Record inserted
PFE Company Details Record inserted
MRK Company Details Record inserted
BAC Company Details Record inserted
KO Company Details Record inserted
TMO Company Details Record inserted
ORCL Company Details Record inserted
COST Company Details Record inserted
AVGO Company Details Record inserted
CSCO Company Details R

KR Company Details Record inserted
AME Company Details Record inserted
OTIS Company Details Record inserted
MTD Company Details Record inserted
RMD Company Details Record inserted
VICI Company Details Record inserted
ODFL Company Details Record inserted
KEYS Company Details Record inserted
SBAC Company Details Record inserted
PPG Company Details Record inserted
ALB Company Details Record inserted
DLR Company Details Record inserted
CPRT Company Details Record inserted
CTSH Company Details Record inserted
ON Company Details Record inserted
ROK Company Details Record inserted
PEG Company Details Record inserted
WEC Company Details Record inserted
GWW Company Details Record inserted
HPQ Company Details Record inserted
DHI Company Details Record inserted
ES Company Details Record inserted
OKE Company Details Record inserted
FAST Company Details Record inserted
BKR Company Details Record inserted
VRSK Company Details Record inserted
GLW Company Details Record inserted
DFS Company Details Re

AOS Company Details Record inserted
FFIV Company Details Record inserted
UHS Company Details Record inserted
TPR Company Details Record inserted
AAP Company Details Record inserted
PNW Company Details Record inserted
HAS Company Details Record inserted
FRT Company Details Record inserted
CMA Company Details Record inserted
IVZ Company Details Record inserted
RHI Company Details Record inserted
FBHS Company Details Record inserted
SBNY Company Details Record inserted
WHR Company Details Record inserted
DISH Company Details Record inserted
SEE Company Details Record inserted
PNR Company Details Record inserted
RL Company Details Record inserted
ZION Company Details Record inserted
NCLH Company Details Record inserted
AIZ Company Details Record inserted
OGN Company Details Record inserted
DVA Company Details Record inserted
XRAY Company Details Record inserted
LNC Company Details Record inserted
DXC Company Details Record inserted
MHK Company Details Record inserted
GNRC Company Details R

### Data Cleaning on API DataFrame

In [67]:
daily_df.head()

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume
0,2022-01-05,AAPL,173.910645,174.919998,180.169998,174.639999,179.610001,94537600
1,2022-01-05,AMZN,164.356995,164.356995,167.126495,164.356995,166.882996,64302000
2,2022-01-05,MSFT,313.442993,316.380005,326.070007,315.980011,325.859985,40054300
3,2022-01-05,TSLA,362.706665,362.706665,390.113342,360.336670,382.216675,80119800
4,2022-01-06,AAPL,171.007523,172.000000,175.300003,171.639999,172.699997,96904000


In [68]:
hourly_df.head()

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume
0,2022-12-05 09:30:00-05:00,AAPL,148.480804,148.480804,150.919907,147.500000,147.770004,22202031.0
1,2022-12-05 09:30:00-05:00,AMZN,92.684998,92.684998,94.059998,92.510002,93.050003,16020869.0
2,2022-12-05 09:30:00-05:00,MSFT,251.580002,251.580002,253.819000,251.070007,252.009995,4548397.0
3,2022-12-05 09:30:00-05:00,TSLA,184.960007,184.960007,191.270004,183.800003,189.100006,31428674.0
4,2022-12-05 10:30:00-05:00,AAPL,146.880005,146.880005,148.750000,146.820007,148.479996,9532365.0


In [69]:
# Fetching number of rows and columns
daily_df.shape

(924, 8)

In [70]:
hourly_df.shape

(32, 8)

In [71]:
# Fetching unique values in each column
daily_df.nunique()

Date         231
Ticker         4
Adj Close    915
Close        906
High         904
Low          907
Open         907
Volume       924
dtype: int64

In [72]:
hourly_df.nunique()

Date          9
Ticker        4
Adj Close    32
Close        32
High         30
Low          31
Open         32
Volume       29
dtype: int64

In [73]:
# Searching for duplicate entries
daily_df.duplicated().sum()

0

In [74]:
hourly_df.duplicated().sum()

0

In [75]:
# Searching for NULL values in the DataFrame
daily_df.isnull().sum()

Date         0
Ticker       0
Adj Close    0
Close        0
High         0
Low          0
Open         0
Volume       0
dtype: int64

In [76]:
hourly_df.isnull().sum()

Date         0
Ticker       0
Adj Close    0
Close        0
High         0
Low          0
Open         0
Volume       0
dtype: int64

In [77]:
# Displaying final DataFrame after Data Cleaning
daily_df.head()

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume
0,2022-01-05,AAPL,173.910645,174.919998,180.169998,174.639999,179.610001,94537600
1,2022-01-05,AMZN,164.356995,164.356995,167.126495,164.356995,166.882996,64302000
2,2022-01-05,MSFT,313.442993,316.380005,326.070007,315.980011,325.859985,40054300
3,2022-01-05,TSLA,362.706665,362.706665,390.113342,360.336670,382.216675,80119800
4,2022-01-06,AAPL,171.007523,172.000000,175.300003,171.639999,172.699997,96904000


In [78]:
hourly_df.head()

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume
0,2022-12-05 09:30:00-05:00,AAPL,148.480804,148.480804,150.919907,147.500000,147.770004,22202031.0
1,2022-12-05 09:30:00-05:00,AMZN,92.684998,92.684998,94.059998,92.510002,93.050003,16020869.0
2,2022-12-05 09:30:00-05:00,MSFT,251.580002,251.580002,253.819000,251.070007,252.009995,4548397.0
3,2022-12-05 09:30:00-05:00,TSLA,184.960007,184.960007,191.270004,183.800003,189.100006,31428674.0
4,2022-12-05 10:30:00-05:00,AAPL,146.880005,146.880005,148.750000,146.820007,148.479996,9532365.0


In [79]:
# Cleaning existing data from the database
web_del_query = 'delete from daily_prices where dp_id is not null'
mycursor.execute(web_del_query)
mydb.commit()

In [80]:
# Cleaning existing data from the database
web_del_query = 'delete from hourly_prices where hp_id is not null'
mycursor.execute(web_del_query)
mydb.commit()

In [81]:
#Iterate into the Daily DataFrame
for i,row in daily_df.iterrows():
    #INSERT query for inserting the data into the database
    daily_price_sql = "INSERT INTO stockassist.daily_prices(dp_date,dp_ticker,dp_adj_close,dp_close,dp_high,dp_low,dp_open,dp_volume) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
    try:
        #Execute the SQL query
        mycursor.execute(daily_price_sql, tuple(row))
        print('Daily Price Record inserted')
        #Commit the executed query
        mydb.commit()

    except:
        #Display message if the record already exists inside the table
        print('Daily Price Record already exists')


Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record i

Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record i

Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record i

Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted
Daily Price Record inserted


In [82]:
#Iterate into the Hourly DataFrame
for i,row in hourly_df.iterrows():
    #INSERT query for inserting the data into the database
    hourly_sql = "INSERT INTO stockassist.hourly_prices(hp_date,hp_ticker,hp_adj_close,hp_close,hp_high,hp_low,hp_open,hp_volume) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
    try:
        #Execute the SQL query
        mycursor.execute(hourly_sql, tuple(row))
        print('Hourly Price Record inserted')
        #Commit the executed query
        mydb.commit()

    except:
        #Display message if the record already exists inside the table
        print('Hourly Price Record already exists')


Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
Hourly Price Record inserted
